In [45]:
DATA_PATH = '../../data/CRTS2/'

In [46]:
import pandas as pd

Load Transient Catalog

In [47]:
filename = 'transient_catalog.pickle'
indir = DATA_PATH; filepath = indir + filename
df_cat = pd.read_pickle(filepath)
print(df_cat.TransientID.unique().shape)

(5539,)


Load Transient Lightcurves

In [48]:
filename = 'transient_lightcurves.pickle'
indir = DATA_PATH; filepath = indir + filename
df_lcs = pd.read_pickle(filepath)
print(df_lcs.TransientID.unique().shape)

(4984,)


Check that all Lightcurves TransientID's are in Catalog

In [49]:
all_exist = True
transID_cat_list = df_cat.TransientID.unique()
for lcs_id in df_lcs.TransientID.unique():
    all_exist = all_exist and (lcs_id in transID_cat_list)
print('All exist:', str(all_exist))

All exist: True


Check that all Catalog TransientID's are in Lightcurves 

In [50]:
all_exist = True
lcs_missing_transientID_list = []
transID_lcs_list = df_lcs.TransientID.unique()
for cat_id in df_cat.TransientID.unique():
    id_exists = (cat_id in transID_lcs_list)
    all_exist = all_exist and id_exists
    if not id_exists: lcs_missing_transientID_list.append(cat_id)
print('All exist:', str(all_exist))
print('Missing: {} Transients'.format(len(lcs_missing_transientID_list)))

All exist: False
Missing: 555 Transients


Merge dataframes with inner join

In [51]:
df_merge = df_cat.copy().merge(df_lcs.copy().groupby('TransientID',as_index=False).count(), how='inner')
df_merge.rename(columns={'Mag':'ObsCount'}, inplace=True)
df_merge = df_merge[['TransientID', 'Classification', 'ObsCount']]

In [52]:
print(df_merge.shape)

(4985, 3)


Top 10 count of transients by class

In [53]:
df = df_merge[['Classification','ObsCount']].groupby('Classification').count()
df = df.rename(columns={'ObsCount':'ObjCount'}).sort_values('ObjCount', ascending=False)
df.head(10).transpose()

Classification,SN,CV,HPM,AGN,SN?,Blazar,Flare,Unknown,AGN?,CV?
ObjCount,1539,943,437,429,294,238,215,186,133,68


Describe transients observation count

In [54]:
df_lcs.groupby('TransientID').count().describe()

,Mag,Magerr,MJD
count,4984.000000,4984.000000,4984.000000
mean,90.584671,90.584671,90.584671
std,112.330606,112.330606,112.330606
min,1.000000,1.000000,1.000000
25%,9.000000,9.000000,9.000000
50%,35.000000,35.000000,35.000000
75%,140.000000,140.000000,140.000000
max,880.000000,880.000000,880.000000


Count number of objects with at least 5 observations

In [55]:
df_merge_filtered = df_merge[df_merge.ObsCount >= 5]
df_merge_filtered.shape[0]

4385

Top 10 count of transients by class (with at least 5 observations)

In [57]:
df = df_merge_filtered[['Classification','ObsCount']].groupby('Classification').count()
df = df.rename(columns={'ObsCount':'ObjCount'}).sort_values('ObjCount', ascending=False)
df.head(20).transpose()

Classification,SN,CV,AGN,HPM,SN?,Blazar,Flare,AGN?,Unknown,CV?,Var,Ast?,SN/CV,SN/AGN,YSO,Mira,Blazar?,Flare?,Ast,Var?
ObjCount,1295,862,427,413,239,236,207,132,112,55,46,31,28,23,21,20,18,18,12,11


Describe observation count for transients with at least 5 observations

In [44]:
df_lcs[df_lcs.TransientID.isin(df_merge_filtered.TransientID)].groupby('TransientID').count().describe()

,Mag,Magerr,MJD
count,4384.000000,4384.000000,4384.000000
mean,102.623631,102.623631,102.623631
std,114.634489,114.634489,114.634489
min,5.000000,5.000000,5.000000
25%,15.000000,15.000000,15.000000
50%,48.000000,48.000000,48.000000
75%,163.250000,163.250000,163.250000
max,880.000000,880.000000,880.000000
